<a href="https://colab.research.google.com/github/virajvaidya/Australia-Energy-Demand-Forecasting/blob/main/SAPIPELINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas==1.3.5

In [ ]:
!pip install plotly==5.4.0
!pip install chart_studio

In [ ]:
!pip install statsmodels==0.13.1

In [ ]:
!pip install pmdarima

In [5]:
import pandas               as pd
import numpy                as np
import pmdarima             as pm
import seaborn              as sns
import plotly.graph_objs    as go

from pmdarima           import pipeline
from pmdarima           import model_selection
from pmdarima           import preprocessing as ppc
from pmdarima           import arima
from matplotlib         import pyplot as plt
from plotly.offline     import iplot
from plotly.subplots    import make_subplots

In [6]:
sns.set_style('whitegrid')
pd.plotting.register_matplotlib_converters()
# Default figure size
sns.mpl.rc('figure',figsize=(16, 6))

In [7]:
from google.colab import files
import io
uploaded = files.upload()

Saving SA.csv to SA.csv


In [8]:
df = pd.read_csv('SA.csv')

In [9]:
df= df[['DATE','TOTALDEMANDSA','RRPSA']]

In [10]:
df['DATE'] = df['DATE'].apply(lambda x: pd.Timestamp(x).strftime('%d/%m/%Y %H%M'))

In [11]:
df['DATE'] = df['DATE'].astype('datetime64[ns]')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70942 entries, 0 to 70941
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   DATE           70942 non-null  datetime64[ns]
 1   TOTALDEMANDSA  70942 non-null  float64       
 2   RRPSA          70942 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1.6+ MB


In [14]:
data = df['TOTALDEMANDSA']
train, test = model_selection.train_test_split(data, train_size=40000)

In [15]:
pipe = pipeline.Pipeline([
    ("fourier", ppc.FourierFeaturizer(m=12, k=4)),
    ("arima", arima.AutoARIMA(stepwise=True, trace=1, error_action="ignore",
                              information_criterion='bic',
                              seasonal=False,  # because we use Fourier
                              suppress_warnings=True))
])

In [16]:
pipe.fit(train)
print("Model fit:")
print(pipe)

Performing stepwise search to minimize bic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : BIC=426552.417, Time=122.44 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : BIC=448296.799, Time=3.91 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : BIC=430547.804, Time=3.73 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : BIC=437281.276, Time=25.19 sec
 ARIMA(0,1,0)(0,0,0)[0]             : BIC=448286.203, Time=31.00 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : BIC=429391.995, Time=23.38 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : BIC=429414.889, Time=23.55 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : BIC=426878.035, Time=161.80 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : BIC=inf, Time=148.64 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : BIC=429600.269, Time=17.67 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : BIC=429400.015, Time=33.74 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : BIC=426817.932, Time=125.17 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : BIC=inf, Time=163.57 sec
 ARIMA(2,1,2)(0,0,0)[0]             : BIC=426541.821, Time=85.61 sec
 A

In [17]:
preds, conf_int = pipe.predict(n_periods=100, return_conf_int=True)
print("\nForecasts:")
print(preds)


Forecasts:
[1144.94824231 1129.46431228 1111.98072087 1099.97503019 1093.65670394
 1104.71713047 1140.84700133 1173.0194233  1176.7790825  1162.97822147
 1142.91440499 1119.94089253 1098.27246277 1075.29607116 1053.63175507
 1040.4047344  1035.41023868 1049.88957726 1091.07088874 1129.47249539
 1140.20884656 1133.73964201 1121.01721262 1105.10315733 1089.97731681
 1072.84885217 1056.21525141 1047.13062956 1045.36533261 1062.17577115
 1104.83830522 1143.94552925 1154.70522863 1147.68281401 1133.94258242
 1116.65896557 1099.92090373 1081.03919712 1062.60266659 1051.74444496
 1048.30038931 1063.57843997 1104.89263521 1142.86008414 1152.70103144
 1144.98266245 1130.76195609 1113.19865749 1096.36137875 1077.53663638
 1059.28659512 1048.71677352 1045.63570404 1061.32533018 1103.07582231
 1141.48317357 1151.74968872 1144.42798108 1130.56386712 1113.30926135
 1096.72806088 1078.10488316 1060.00245001 1049.5288774  1046.49692064
 1062.19395415 1103.91636513 1142.26679325 1152.45430948 1145.038